In [1]:
import os
import random

os.environ['KERAS_BACKEND'] = "tensorflow"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
from keras.utils.io_utils import HDF5Matrix
import h5py
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.models import Model
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [2]:
# DEFINES
TRAIN_PATH = 'food-images/food_c101_n10099_r64x64x3.h5'
TEST_PATH = 'food-images/food_test_c101_n1000_r64x64x3.h5'

IMAGE_CHANNELS = 3
BATCH_SIZE = 32
EPOCHS = 100
IMAGE_WIDTH, IMAGE_HEIGHT = 3, 3
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)

In [3]:
# GET DATA
X_train = HDF5Matrix(TRAIN_PATH, 'images')  # Shape (10099, 32, 32, 3) => [image_index][pixel_width_index][pixel_height_index][color_channel_index]
y_train = HDF5Matrix(TRAIN_PATH, 'category')  # Shape (1000, 101) => [image_index][one_hot_encoded_vector]
print("X_train shape: ", X_train.shape, " y_train shape: ", y_train.shape)

X_test = HDF5Matrix(TEST_PATH,
                    'images')  # Shape (10099, 32, 32, 3) => [image_index][pixel_width_index][pixel_height_index][color_channel_index]
y_test = HDF5Matrix(TEST_PATH, 'category')  # Shape (1000, 101) => [image_index][one_hot_encoded_vector]
print("X_test shape: ", X_test.shape, " y_test shape: ", y_test.shape)

X_train shape:  (10099, 64, 64, 3)  y_train shape:  (10099, 101)
X_test shape:  (1000, 64, 64, 3)  y_test shape:  (1000, 101)


In [4]:
base_model = VGG16(include_top = False, weights='imagenet', input_tensor=None, input_shape = [64, 64, 3], pooling=None, classes=1000)
#base_model = Xception(include_top= False, weights='imagenet', input_tensor=None, input_shape = [384, 384, 3], pooling=None, classes=1000)
#base_model = InceptionV3(include_top = False, weights = 'imagenet', input_tensor = None, input_shape = [384, 384, 3], pooling = None, classes = 101)
# Pop the last dense layer
#base_model.layers.pop()
# Form a new model with just the layers we meed
filtered_base_model = Model(inputs = base_model.input, outputs = base_model.layers[-1].output)

# Add my layer[s]
output = filtered_base_model.output
output = Flatten()(output)
output = Dense(1024, name="1-dense", activation='relu')(output)
output = Dense(2048, name="12-dense", activation='relu')(output)
output = Dense(101, activation = 'softmax')(output)


classifier = Model(inputs=base_model.input, outputs=output)
    
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

58736640/58889256 [============================>.] - ETA: 0s

In [ ]:
print( classifier.summary() )


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [ ]:
classifier.fit(X_train, 
               y_train, 
               batch_size= 32,
               epochs = 80,
               shuffle="batch"
)

Epoch 1/80
10099/10099 [==============================] - 71s - loss: 15.9365 - acc: 0.0109    
Epoch 2/80
10099/10099 [==============================] - 65s - loss: 15.9457 - acc: 0.0107    
Epoch 3/80
10099/10099 [==============================] - 68s - loss: 15.9457 - acc: 0.0107    
Epoch 4/80
10099/10099 [==============================] - 70s - loss: 15.9457 - acc: 0.0107    
Epoch 5/80
10099/10099 [==============================] - 69s - loss: 15.9457 - acc: 0.0107    
Epoch 6/80
10099/10099 [==============================] - 69s - loss: 15.9457 - acc: 0.0107    
Epoch 7/80
10099/10099 [==============================] - 69s - loss: 15.9457 - acc: 0.0107    
Epoch 8/80
 7392/10099 [====================>.........] - ETA: 18s - loss: 15.9393 - acc: 0.0111